# Data Integration with CHAKRA

## Import CHAKRA library

In [1]:
using Chakra
#include("src/Chakra.jl")
#using Main.Chakra

## Implement an interface for a data source S1

The interface must define the core CHAKRA types and operations.

In [2]:
module S1
    using Main.Chakra
    Chakra.@Reference Int []
    
    # DEFINE ATTRIBUTES

    @DefineAtt(:A,Int)

    data = ins(Id(1),seta(Att(:A),10,agg(Id)),emp(Hierarchy))

end

Main.S1

In [3]:
typeof(S1.data)

Main.S1.Hierarchy

In [4]:
dom(S1.data)

1-element Vector{Main.S1.Id}:
 Main.S1.Id(1)

In [5]:
cts(S1.data)

1-element Vector{Pair{Main.S1.Id, Main.S1.Constituent}}:
 Main.S1.Id(1) => Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [6]:
c1 = fnd(S1.Id(1),S1.data)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [7]:
fnd(S1.Id(2),S1.data)

Main.Chakra.None()

In [8]:
methods(S1.__atyp__)[2:end-1]

[1] __atyp__(::Main.S1.AttName{:A}) in Main.S1 at /Users/nick/Dropbox/Chakra/src/Chakra.jl:299

In [9]:
geta(:A,c1)

10

In [10]:
geta(:A,S1.Id(1),S1.data)

10

In [11]:
geta(:A,S1.Id(2),S1.data)

Main.Chakra.None()

In [12]:
geta(:B,c1)

LoadError: Attribute B is not defined in module Main.S1.

## Implement a second data source interface S2

In [13]:
module S2
    using Main.Chakra
    Chakra.@Reference Int []

    @DefineAtt(:B,String)

    data = ins(Id(2),seta(Att(:B),"B Value",agg(Id)),emp(Hierarchy))

end

Main.S2

In [14]:
dom(S2.data)

1-element Vector{Main.S2.Id}:
 Main.S2.Id(2)

In [15]:
c2 = fnd(S2.Id(2),S2.data)

Main.S2.Constituent(Dict{Symbol, Any}(:B => "B Value"), Dict{Symbol, Any}(), Main.S2.Id[])

In [16]:
geta(:B,c2)

"B Value"

## Link data sources S1 and S2 using a third data source S3

In [17]:
module S3
    using Main.Chakra
    using Main.S1, Main.S2

    Chakra.@Reference Int [S1,S2]

    @DefinePrp(:TYPE,String)
    
    Chakra.fnd(x::S1.Id,h::Hierarchy) = fnd(x,S1.data)
    Chakra.fnd(x::S2.Id,h::Hierarchy) = fnd(x,S2.data)
    Chakra.dom(h::Hierarchy) = vcat(reverse(collect(keys(h.constituents))),dom(S2.data),dom(S1.data))
    Chakra.cts(h::Hierarchy) = vcat(reverse(collect(h.constituents)),cts(S2.data),cts(S1.data))

    data = ins(Id(3),setp(Prp(:TYPE),"Rel",agg(DOMAIN[S1.Id(1),S2.Id(2)])),emp(Hierarchy))

end

Main.S3

In [18]:
cts(S3.data)

3-element Vector{Pair{Any, Any}}:
 Main.S3.Id(3) => Main.S3.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(:TYPE => "Rel"), Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}[Main.S1.Id(1), Main.S2.Id(2)])
 Main.S2.Id(2) => Main.S2.Constituent(Dict{Symbol, Any}(:B => "B Value"), Dict{Symbol, Any}(), Main.S2.Id[])
 Main.S1.Id(1) => Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [19]:
c3 = fnd(S3.Id(3),S3.data)

Main.S3.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(:TYPE => "Rel"), Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}[Main.S1.Id(1), Main.S2.Id(2)])

In [20]:
fnd(S1.Id(1),S3.data)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [21]:
ps = pts(c3)

2-element Vector{Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}}:
 Main.S1.Id(1)
 Main.S2.Id(2)

In [22]:
s = sequence(ps,S3.data)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(:B => "B Value"), Dict{Symbol, Any}(), Main.S2.Id[])

In [23]:
sequence(S3.Id(3),S3.data)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(:B => "B Value"), Dict{Symbol, Any}(), Main.S2.Id[])

In [24]:
x = gethead(s)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [25]:
geta(:A,x)

10